## Massive Nearest Neighbors Using K-D-Trees

## The problem we are presenting

  - Finding the N nearest neighbors to a point from a set of M points in any dimension requires evaluating all the distances and storing the N best.
  - We talk about "massive" search when we are trying to find the nearest neighbor not to one point, but to all the M points (or another huge set of points).
  
This is a huge (solved) problem in Geographical Information Systems and has endless applications including: completing missing information, creating valuable features, clustering, ...

## Bad approaches

  - In 1-D the problem is trivial, we just sort the points and we are done.
  - Our problem is not being able to "sort points" in D != 1 to do the same. Of course, we can sort the points, but any order is irrelevant to our distance problem.
  - Brute-forcing the problem solves it, but is M^2. This is usually not feasible. For 10 million points (a normal number for housing in Spain) we compute 100 trillion distances and that is not feasible.
  

## K-D Trees

In computer science, a k-d tree (short for k-dimensional tree) is a space-partitioning data structure for organizing points in a k-dimensional space. k-d trees are a useful data structure for several applications, such as searches involving a multidimensional search key (e.g. range searches and nearest neighbor searches). k-d trees are a special case of binary space partitioning trees.

https://en.wikipedia.org/wiki/K-d_tree

## Informal description

The k-d tree is a binary tree in which every leaf node is a k-dimensional point. Every non-leaf node can be thought of as implicitly generating a splitting hyperplane that divides the space into two parts, known as half-spaces. Points to the left of this hyperplane are represented by the left subtree of that node and points to the right of the hyperplane are represented by the right subtree. The hyperplane direction is chosen in the following way: every node in the tree is associated with one of the k dimensions, with the hyperplane perpendicular to that dimension's axis. So, for example, if for a particular split the "x" axis is chosen, all points in the subtree with a smaller "x" value than the node will appear in the left subtree and all points with larger "x" value will be in the right subtree. In such a case, the hyperplane would be set by the x-value of the point, and its normal would be the unit x-axis.

https://en.wikipedia.org/wiki/K-d_tree

## Make a matrix of 10000 normally random numbers in 3D

In [1]:
import numpy as np
from scipy.spatial import KDTree

matrix = np.random.normal(0, 10, (10000, 3))


In [2]:
matrix

array([[ -8.87505009,   7.92356168,  21.84247434],
       [  9.43285392,   4.11431052,  10.1281878 ],
       [  3.57115085,   6.94308926,  -9.26330703],
       ...,
       [ -5.26766407,   8.47978473,  -2.30757196],
       [  3.32507109, -16.18184459,   0.25392537],
       [  1.71288849,  13.56166051,   7.89188364]])

## Caveat !! <font color='red'>This array is not copied, and so modifying this data will result in bogus results.</font>

## Create the K-D tree

In [3]:
kdtree = KDTree(matrix)

## Query the 10 nearest neighbors of all the points

In [4]:
d, index = kdtree.query(matrix, k=10)

The distance is an numpy.ndarray. Since the set we are querying is the same set, as expected the first distance is always zero.

In [5]:
d

array([[0.        , 2.22057007, 2.35368339, ..., 3.91599897, 4.02567513,
        4.07526298],
       [0.        , 0.19044082, 1.43652236, ..., 1.92504645, 1.96195625,
        1.99163441],
       [0.        , 0.75370343, 1.08611599, ..., 1.50165193, 1.51942635,
        1.54146202],
       ...,
       [0.        , 0.99325511, 1.32777377, ..., 1.92283463, 1.97637655,
        2.13292684],
       [0.        , 1.04207364, 1.69654158, ..., 2.33580621, 2.42255518,
        2.52474483],
       [0.        , 0.96153578, 1.21464942, ..., 1.85300812, 2.01278298,
        2.02482244]])

The index, another numpy.ndarray, is the row in matrix identifying the neighbor. Again, since it is the same set, the nearest to each one is itself.

In [6]:
index

array([[   0, 7364, 4494, ..., 5617, 1036, 4203],
       [   1, 9187, 9331, ..., 7235, 3077, 4094],
       [   2, 4802, 3277, ...,  808, 9899,  705],
       ...,
       [9997,  864, 6218, ..., 8219, 4934, 9296],
       [9998, 8533, 8692, ..., 4663,  624, 8223],
       [9999, 9348,  653, ..., 7361, 6497, 6874]])

## We can also compute distances to another set (in the same dimension, of course)

In [7]:
another_matrix = np.random.normal(0, 5, (500, 3))
another_matrix

array([[ 7.45933333,  1.27580632, -6.21123837],
       [ 6.94104365, -7.29066376, -8.61381681],
       [-2.73004431, -3.49417461,  2.99591107],
       ...,
       [ 4.05303656,  0.65910546,  4.32288907],
       [ 4.71662553,  6.48927328,  2.81568275],
       [-3.65901424, -2.05872426, -4.38814954]])

In [8]:
d, index = kdtree.query(another_matrix, k=5)

In [9]:
d.shape

(500, 5)

We can set a new number of nearest neighbors, only dimension cannot change. The results have as many rows as the query matrix, containing indices from the original matrix.

In [10]:
index

array([[6501, 8577, 6169,  757, 9913],
       [2865, 9413, 7526, 1795,  991],
       [6007, 8321, 2485, 7513, 2388],
       ...,
       [7581, 7528, 4064,  227, 1776],
       [8500,   69, 2344,  154, 5932],
       [3883, 7747, 6155, 4486, 2864]])

## A c++ top performance implementation using shared memory multi-threading

### R package RANN

Computes 10 million rows datasets in seconds on a laptop.

https://cran.r-project.org/web/packages/RANN/index.html